<a href="https://colab.research.google.com/github/mjvakili/MLcourse/blob/master/day5/Gaussian_mixture_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gaussian mixture model

Let’s imagine a random variable $x \in R^{D}$ ($x$ lives in a $D$-dimensional Eucleadian space.) $x$ is said to be drawn from a mixture of Gaussians if:

$p(x) = \sum_{k=1}^{K} \pi_k \mathcal{N}(x|\mu_k, \Sigma_k)$,

Where $\mu_k$ is a $D$-dimensional vector, $\Sigma_k$ is a $D\times D$ matrix, $\pi_k$ is a positive scalar such that $\sum_{k=1}^{K} \pi_{k} = 1$ and $k = 1,...,K$.  

Intuitively speaking, one can think of such model K clusters, each following a Gaussian distribution. The mixture coefficients $\{\pi_k\}_{k=1}^{K}$ are the cluster prior probabilities, and hence they sum to unity. The probability that a given data point $\tilde{x}$ belongs to a cluster cluster $\tilde{k}$ is: 

$p(cluster = \tilde{k}| x)$
 $= \frac{\pi_\tilde{k} \mathcal{N}(x|\mu_\tilde{k}, \Sigma_\tilde{k})}{\sum_{k=1}^{K} \pi_k \mathcal{N}(x|\mu_k, \Sigma_k)}$


So unlike the K-means algorithms, there is no hard assignment of clusters to the data-points and the cluster assignment is probabilistic.
This is shown in the following example:

Let’s consider an iid dataset : $\{x_n\}_{n=1}^{N}$:

$p(\{x_n\}_{n=1}^{N} | ...) = \prod_{n=1}^{N} p(x_n | ….)$

Therefore, 

$p(\{x_n\}_{n=1}^{N} | ...) = \prod_{n=1}^{N} \sum_{k=1}^{K} \pi_k \mathcal{N}(x_n|\mu_k, \Sigma_k)$

Note that finding the maximum likelihood solution for such a problem is not possible. If any of the Gaussian clusters collapses on a data-point $x_{m}$ (that is $\mu_{k}$ = $x_{m}$ and $\Sigma_{k} = 0_{D\times D}$) then the above probability becomes infinity. 

In practice, 
The parameters of the model $\{\mu_k , \Sigma_k, \pi_k\}_{k=1}^{K}$ are estimated by the Expectation-Maximization (EM) algorithm. We do not discuss the details of the algorithm in this lecture but you can learn about them in  the following references. 


#How many mixtures of Gaussians should I be using for my problem?

This is another instance of model complexity. Choosing a large number of clusters can yeild a higher likelihood given the training data but at the expense of using an unnecessarily complex model. One way to tackle this issue is to make use of the information criteria. The information criteria encourage higher likelihood of the data but at the same time punishes higher model complexity. 

Two common choices of IC are alkaik information criteria (AIC) 

AIC = $-2 \log \mathcal{L} + N_{data}$

And Bayesian information criteria (BIC)

BIC =  $-2 \log \mathcal{L} + N_{data} \log N_{params}$
